# *Feature Engineering* - Tratamento dos dados brutos

<div>
<img src="imagens/ff.jpg" align="left" width="400"/>
</div>

---
### Rodrigo Fragoso 
- [**Linkedin**](https://www.linkedin.com/in/rodrigo-a-fragoso/) <br/>
- **Email** : rodrigoandradefragoso@gmail.com <br/>

### Resumo
-  Com uma quantidade massiva e desorganizada de dados, é necessário realizar algumas transformações para torná-los consumíveis nos modelos de *Machine Learning* e até melhorar a sua performance ;
-  O output esperado é um *dataframe* organizado, com seus dados tratados e limpos.
---

<a id='top'></a>
## Sumário

* [1 - Importações das bibliotecas](#t1)

* [2 - Tratamento e criação de features](#t2)
    * [2.1 - Seleção das variáveis brutas](#t2_1)
    * [2.2 - Transformação das variáveis categóricas](#t2_2)
    * [2.3 - Transformação das variáveis númericas divididas entre *men* and *women*](#t2_3)
    * [2.4 - Transformação das variáveis particulares](#t2_4)
    * [2.5 - Variáveis descartadas devido ao excesso de nulos](#t2_5)<br>
<br>   
* [3 - Verificação dos resultados](#t3)

##     

<a id='t1'></a>
## 1 - Importações das bibliotecas
- [Sumário](#top)   
    - [Próximo](#t2)

### Para iniciarmos a extração será necessário o uso de algumas bibliotecas específicas, que serão importadas na célula abaixo:
-  Pandas: ferramenta rápida e poderosa, responsável pela manipulação/analise de dados através do formato *dataframe* ;
-  re: modulo para realizar operações de correspondência (em texto) através de expressões regulares ;
- time: modulo utilizado, principalmente, para cálculo de tempo de processamento e criação de *delays* ;

In [1]:
import pandas as pd
import numpy as np
import re
import time

import json

##     

<a id='t2'></a>
## 2 - Tratamento e criação de features
- [Sumário](#top) 
    - [Anterior](#t1)
    - [Próximo](#t2_1)

In [2]:
df = pd.read_json("./dados_json/parsed_shoes_info.json", lines=True)
df.shape

(2277, 744)

### O primeiro passo é carregar os dados e verificar se as dimensões estão coerentes:

* Como estamos trabalhando com muitas colunas, é necessário ajustar o pandas para que seja possível visualizar todas elas. Isto pode ser feito utilizando o set_option( ) com os argumentos ('display.max_columns', None), aqui utilizamos o **None** para visualizar todas as colunas do *dataset* mas também é possível selecionar um número inteiro especifico, caso seja necessário:

In [3]:
pd.set_option('display.max_columns', None)
df.head(4)

A popular pick  A top 1% best Road running shoe  \
0  rating-fact-bar-value-6             NaN                              NaN   
1  rating-fact-bar-value-6             NaN                              NaN   
2  rating-fact-bar-value-6             NaN                              NaN   
3  rating-fact-bar-value-8             NaN                              NaN   

   A top 1% best Trail running shoe  A top 10% best Road running shoe  \
0                               NaN                               NaN   
1                               NaN                               NaN   
2                               NaN                               NaN   
3                               NaN                               NaN   

   A top 10% best Trail running shoe  A top 2% best Road running shoe  \
0                                NaN                              NaN   
1                                NaN                              NaN   
2                                NaN                              NaN   
3                                NaN                              NaN   

   A top 2% best Trail running shoe  A top 3% best Road running shoe  \
0                               NaN                              NaN   
1                               NaN                              NaN   
2                               NaN                              NaN   
3                               NaN                              NaN   

   A top 3% best Trail running shoe  A top 4% best Road running shoe  \
0                               NaN                              NaN   
1                               NaN                              NaN   
2                               NaN                              NaN   
3                               NaN                              NaN   

   A top 4% best Trail running shoe  A top 5% best Road running shoe  \
0                               NaN                              NaN   
1                               NaN                              NaN   
2                               NaN                              NaN   
3                               NaN                              NaN   

   A top 5% best Trail running shoe  A top 6% best Road running shoe  \
0                               NaN                              NaN   
1                               NaN                              NaN   
2                               NaN                              NaN   
3                               NaN                              NaN   

   A top 6% best Trail running shoe  A top 7% best Road running shoe  \
0                               NaN                              NaN   
1                               NaN                              NaN   
2                               NaN                              NaN   
3                               NaN                              NaN   

   A top 7% best Trail running shoe  A top 8% best Road running shoe  \
0                               NaN                              NaN   
1                               NaN                              NaN   
2                               NaN                              NaN   
3                               NaN                              NaN   

   A top 8% best Trail running shoe  A top 9% best Road running shoe  \
0                               NaN                              NaN   
1                               NaN                              NaN   
2                               NaN                              NaN   
3                               NaN                              NaN   

   A top 9% best Trail running shoe  A top rated 361 Degrees running shoe  \
0                               NaN                                   1.0   
1                               NaN                                   NaN   
2                               NaN                                   NaN   
3                               NaN                              

##     

<a id='t2_1'></a>
## 2.1 - Seleção das variáveis brutas
- [Sumário](#top) 
    - [Anterior](#t2)
    - [Próximo](#t2_2)

### Apesar do pandas ter sido ajustado, para seleção também foi criado um arquivo em **csv** com a lista de todas colunas existentes, facilitando a busca através de algum editor de texto convencional:

In [4]:
pd.DataFrame({'colunas':df.columns}).to_csv('colunas.csv', index= False)

### Através da exploração visual de algumas amostras do nosso conjunto, algumas variáveis foram pré-selecionadas para serem tratadas, transformadas e até servirem de insumo para criação de outras

---
## Lista de variáveis selecionadas

* **shoes_name** : nome comercial do tênis
* **link** : link para página
* **image** : link da imagem do produto
* **terrain-value** : tipo de terreno adequado
* **arch-support-value** : tipo de mecanismo para corrigir o contato entre o arco do pé e o tênis
* **weight-value** : peso (diferenciado entre feminino e masculino) em **g**
* **heel_to_toe_drop** : quedra entre o calcanhar e o pé em **mm**
* **fit-value** : informações adicionais sobre o formato do tênis
* **pronation-value** :  recomendado para o X tipo de pisada
* **arch-type-value** :  recomendado para o X tipo de arco do pé
* **use-value** : modalidade de uso adequeada
* **material-value** : material utilizado para confeccção
* **features-value** : informações especificas do produto
* **strike-pattern-value** : recomendado para quem inicia a pisada com o X espaço do pé
* **foot-condition-value** : tipo de pé do comprador
* **distance-value** : recomendado para distância X
* **technology-value** : tecnologias utilizadas
* **forefoot-height-value** : altura do antepé **mm**
* **heel-height-value** : altura do salto **mm**
* **release-date-value** : data de lançamento
* **brand-value** : marca do produto
* **type-value** : tipo de arco recomendado
* **width-value** : estilo da largura
* **price-value** : preço estimado em BRL
* **Stability** : indicador de estabilidade (0 a 10)
* **Comfort** : indicador de conforto (0 a 10)
* **Durability** : indicador de durabilidade (0 a 10)
* **Traction** : indicador de tração (0 a 10)
* **rr-reviews-score-average** : score (0 a 100) e quantidade de reviews feito por profissionais
* **reasons_not_to_buy** : quantidade de razões para comprar
* **reasons_to_buy** : quantidade de razões para não comprar
* **bad_reasons_to_buy** : Texto explicativo ressaltando as razões para não comprar
* **good_reasons_to_buy** : Texto explicativo ressaltando as razões para comprar

---
### Com a lista pronta, foi criado um novo conjunto de dados apenas com as variáveis pertencentes a lista acima:

In [5]:
df_teste=df[['shoes_name','link','image','terrain-value','arch-support-value','weight-value','heel-to-toe-drop-value','fit-value','pronation-value',
             'arch-type-value','use-value','material-value','features-value','strike-pattern-value','foot-condition-value','distance-value',
             'technology-value','forefoot-height-value','heel-height-value','release-date-value','brand-value','type-value','width-value','price-value',
            'Stability','Comfort','Durability','Traction','rr-reviews-score-average','reasons_not_to_buy','reasons_to_buy',
            'bad_reasons_to_buy','good_reasons_to_buy'
            ]].copy()
df_teste.shape

(2277, 33)

### Obs: As descrições das variáveis foram feitas pela interpretação pessoal do autor deste notebook

##     

<a id='t2_2'></a>
## 2.2 - Transformação das variáveis categóricas
- [Sumário](#top) 
    - [Anterior](#t2_1)
    - [Próximo](#t2_3)

### Vale ressaltar que todas as informações extraídas estão, originalmente, como um objeto de texto. Desse modo, elas foram divididas em 3 grupos:
### - O primeiro são o das Variáveis Categóricas. Por se de tratar de um texto puro, o tênis que possuir mais de uma categoria naquela feature, estaria com elas na mesma *string*
### - Para resolver este problema, foram criadas duas funções: a analise_pre e transform_col_txt, que atuam da seguinte maneira:
* **analise_pre( )**:
    1. É feito um value_counts() da nossa coluna alvo, para entender como aquelas categorias estão escritas ;
    2. Calcula a quantidade de nulos para possível descarte ;
    3. Utilizamos a função split() para dividir as categorias que tenham nomes compostos (ao invés de pegar apenas palavra por palavra) e criamos uma lista, sem valores duplicados, dos grupos ali existentes ;
    4. Realiza-se um print() na tela as categorias existentes e quantas são ;
    5. Exclui a coluna original.
    * *As funções do passo 4 e 5 podem ser ativadas ou não*<br>  
<br>  
* **transform_col_txt( )**:
    1. Realiza os 4 primeiros passos da função anterior ;
    2. Feito isto, é utilizada a técnica de *encondig* conhecida como *one hot*, o *label enconding* não foi utilizado, pois ele acaba atribuindo valores maiores para algumas categorias e com isso, alguns modelos podem dar uma importância maior para estes altos valores, o que geralmente não faz sentido. O nome das novas colunas é adicionado com um prefixo escolhido pelo usuário ;
    3. Exclui a coluna original ;<br> 
   
    <br> 
    <img src="imagens/one_hot.jpg" alt="Drawing" style="width: 400px;"/><br>
    * *Exemplificação das técnicas de enconding mencionadas* <br>
    <br>
    * Obs: a função strip( ) é responsável por remover espaços duplicadas, que estejam a esquerda ou que estejam a direita do texto.
    
### Em relação aos registros nulos (NaN), eles apenas ficariam sem nenhuma categoria (todas preenchidas com 0 no processo manual de *one hot enconding*)

##     

In [6]:
def analise_pre(feature,df,mostrar_valores,drop_col_original):
    print('Valores:')
    print(df[feature].value_counts().head(7))
    print('...')
    print(' ')
    print('Existe(m) {n} valor(es) nulo(s)'.format(n=(df[feature].isna().sum())))

    palavras=[]
    for tipos in df[feature].value_counts().index:
        nivel_1= tipos.replace('|',',').split(',')
        for nivel_2 in nivel_1:
            palavras.append(nivel_2.strip())
    
    palavras_2=[feature]
    for i in palavras: 
        if i not in palavras_2: 
            palavras_2.append(i) 
            
    if mostrar_valores == 1:
        print(' ')
        print('Temos os seguintes valores nesta variável:')
        print(palavras_2[1:])
        print(' ')
        print('Existem {n} possibilidades de features a serem adicionadas'.format(n=len(palavras_2)))
    
    if drop_col_original ==1:
        df.drop(columns=[feature],inplace=True)

In [7]:
def transform_col_txt(feature,df,prefixo,drop_col_original):
    print('Valores:')
    print(df[feature].value_counts().head(7))
    print('...')
    print(' ')
    print('Existe(m) {n} valor(es) nulo(s)'.format(n=(df[feature].isna().sum())))

    palavras=[]
    for tipos in df[feature].value_counts().index:
        nivel_1= tipos.replace('|',',').split(',')
        for nivel_2 in nivel_1:
            palavras.append(nivel_2.strip())
    
    palavras_2=[feature]
    for i in palavras: 
        if i not in palavras_2: 
            palavras_2.append(i) 
    
    print(' ')
    print('Temos os seguintes valores nesta variável:')
    print(palavras_2[1:])
    print(' ')
    
    for tipos in palavras_2[1:]:
        col= palavras_2[0]
        df[prefixo+tipos]=df[col].fillna('nulo').apply(lambda row: 1 if tipos in row else 0) 
    
    if drop_col_original ==1:
        df.drop(columns=[feature],inplace=True)
    
    print(len(palavras_2)-1,' features(s) criada(s) com sucesso')

In [8]:
transform_col_txt('terrain-value',df_teste,'terrain_',1)

Valores:
Road                1581
Trail                599
Road,  Treadmill      62
Mud,  Trail           10
Trail,  Mud            8
Road,  Trail           3
Snow,  Trail           3
Name: terrain-value, dtype: int64
...
 
Existe(m) 1 valor(es) nulo(s)
 
Temos os seguintes valores nesta variável:
['Road', 'Trail', 'Treadmill', 'Mud', 'Snow']
 
5  features(s) criada(s) com sucesso


In [9]:
transform_col_txt('arch-support-value',df_teste,'arch_',1)

Valores:
Neutral           1909
Stability          322
Motion control      23
Name: arch-support-value, dtype: int64
...
 
Existe(m) 23 valor(es) nulo(s)
 
Temos os seguintes valores nesta variável:
['Neutral', 'Stability', 'Motion control']
 
3  features(s) criada(s) com sucesso


In [10]:
transform_col_txt('pronation-value',df_teste,'',1)

Valores:
Neutral Pronation       1909
Overpronation            322
Severe overpronation      23
Name: pronation-value, dtype: int64
...
 
Existe(m) 23 valor(es) nulo(s)
 
Temos os seguintes valores nesta variável:
['Neutral Pronation', 'Overpronation', 'Severe overpronation']
 
3  features(s) criada(s) com sucesso


In [11]:
transform_col_txt('arch-type-value',df_teste,'archtype_',1)

Valores:
High arch      1909
Medium arch     322
Low arch         23
Name: arch-type-value, dtype: int64
...
 
Existe(m) 23 valor(es) nulo(s)
 
Temos os seguintes valores nesta variável:
['High arch', 'Medium arch', 'Low arch']
 
3  features(s) criada(s) com sucesso


In [12]:
transform_col_txt('use-value',df_teste,'use_',1)

Valores:
Jogging                               1295
All-day wear  |  Jogging               209
All-day wear                           108
Fell running  |  Jogging                27
All-day wear,  Walking  |  Jogging      15
All-day wear,  Walking                  10
Triathlon                               10
Name: use-value, dtype: int64
...
 
Existe(m) 574 valor(es) nulo(s)
 
Temos os seguintes valores nesta variável:
['Jogging', 'All-day wear', 'Fell running', 'Walking', 'Triathlon', 'Obstacle course racing']
 
6  features(s) criada(s) com sucesso


In [13]:
transform_col_txt('brand-value',df_teste,'brand_',1)

Valores:
Asics          285
Nike           235
Adidas         226
New Balance    183
Saucony        122
Reebok         120
Brooks         119
Name: brand-value, dtype: int64
...
 
Existe(m) 1 valor(es) nulo(s)
 
Temos os seguintes valores nesta variável:
['Asics', 'Nike', 'Adidas', 'New Balance', 'Saucony', 'Reebok', 'Brooks', 'Salomon', 'Under Armour', 'Mizuno', 'Puma', 'Hoka One One', 'Merrell', 'Altra', 'Inov-8', 'Skechers', 'Newton', 'On', 'The North Face', 'La Sportiva', 'Topo Athletic', 'Zoot', '361 Degrees', 'Vibram FiveFingers', 'Scott', 'Dynafit', 'Vivobarefoot', 'Jordan', 'Salming', "Arc'teryx", 'Xero Shoes', 'Icebug', 'Columbia']
 
33  features(s) criada(s) com sucesso


In [14]:
transform_col_txt('type-value',df_teste,'type_',1)

Valores:
Heavy |  Big guy                     636
Low drop                             268
Low drop |  Zero drop                 81
Heavy |  Big guy |  Low drop          32
Low drop |  Maximalist                30
Low drop |  Zero drop |  Barefoot     26
Heavy |  Big guy |  Maximalist        16
Name: type-value, dtype: int64
...
 
Existe(m) 1117 valor(es) nulo(s)
 
Temos os seguintes valores nesta variável:
['Heavy', 'Big guy', 'Low drop', 'Zero drop', 'Maximalist', 'Barefoot', 'Minimalist']
 
7  features(s) criada(s) com sucesso


In [15]:
transform_col_txt('width-value',df_teste,'width_',1)

Valores:
Normal  |  Normal                           1670
Normal,  Wide  |  Normal,  Wide              137
Normal  |  Normal,  Wide                     109
Normal,  Wide,  X-Wide  |  Normal,  Wide      48
Narrow,  Normal  |  Normal                    48
Normal,  Wide  |  Normal                      35
Normal  |  Narrow,  Normal                    28
Name: width-value, dtype: int64
...
 
Existe(m) 1 valor(es) nulo(s)
 
Temos os seguintes valores nesta variável:
['Normal', 'Wide', 'X-Wide', 'Narrow']
 
4  features(s) criada(s) com sucesso


In [16]:
transform_col_txt('strike-pattern-value',df_teste,'strike_',1)

Valores:
Midfoot strike     832
Heel strike        821
Forefoot strike    405
Name: strike-pattern-value, dtype: int64
...
 
Existe(m) 219 valor(es) nulo(s)
 
Temos os seguintes valores nesta variável:
['Midfoot strike', 'Heel strike', 'Forefoot strike']
 
3  features(s) criada(s) com sucesso


In [17]:
transform_col_txt('distance-value',df_teste,'dist_',1)

Valores:
Daily running |  Long distance |  Marathon                     1537
Competition                                                     562
Daily running |  Marathon                                        23
Daily running |  Long distance |  Ultra running |  Marathon       4
Competition |  Ultra running                                      1
Name: distance-value, dtype: int64
...
 
Existe(m) 150 valor(es) nulo(s)
 
Temos os seguintes valores nesta variável:
['Daily running', 'Long distance', 'Marathon', 'Competition', 'Ultra running']
 
5  features(s) criada(s) com sucesso


##     

<a id='t2_3'></a>
## 2.3 - Transformação das variáveis númericas divididas entre *men* and *women*
- [Sumário](#top) 
    - [Anterior](#t2_2)
    - [Próximo](#t2_4)

### Existem algumas variaveis (em texto) que tem os seus valores numéricos sempre precedidos por *Men* ou *Women*.
### Com o auxilio do biblioteca **re**, foi criada uma função para extrair os valores dessa *string* que possui a seguinte lógica:

* **men_women( )**:
    1. Preenche os valores nulos com -1 ;
    2. Procura por um valor numérica que esteja entre a palavra *Men*/*Women* e a medida escolhida ;
    3. Caso este valor exista, ele é transformado em inteiro e inserido em uma nova coluna que possui o nome da *feature* + o prefixo _*men* ou _*women* ;
    4. Os valores nulos permanecem como -1;
    5. A coluna original é excluída.
    
### Existem diversas técnicas de preenchimentos para os valores nulos.
### O preenchimento por -1 foi escolhido por se tratar de um valor completamente diferente da amostra (negativo) no qual os modelos, provavelmente, possam identificar que ele é totalmente anômalo. Também seria interessante testar *inputers* simples e até mais complexos ou até criar uma nova coluna para indicar que aquela variável possui um valor faltante, mas a primeira foi escolhida por geralmente se mostrar mais simples e tão eficaz quanto as outras.
 

##     

In [18]:
def men_women(feature,df_teste,medida):
    
    df_teste[feature+'_men']=df_teste[feature].fillna(-1).apply(
        lambda row: -1 if row == -1 else (int(re.findall('Men: (\d+)'+medida,row)[0]) if re.findall('Men: (\d+)'+medida,row)!=[] else -1))
    
    df_teste[feature+'_women']=df_teste[feature].fillna(-1).apply(
        lambda row: -1 if row == -1 else (int(re.findall('Women: (\d+)'+medida,row)[0]) if re.findall('Women: (\d+)'+medida,row)!=[] else -1))
    
    df_teste.drop(columns=feature,inplace=True)

In [19]:
analise_pre('forefoot-height-value',df_teste,0,0)
men_women('forefoot-height-value',df_teste,'mm')

Valores:
Men: 18mm |  Women: 18mm    162
Men: 19mm |  Women: 19mm    126
Men: 20mm |  Women: 20mm    118
Men: 17mm |  Women: 17mm    112
Men: 22mm |  Women: 22mm     74
Men: 16mm |  Women: 16mm     71
Men: 12mm |  Women: 12mm     61
Name: forefoot-height-value, dtype: int64
...
 
Existe(m) 784 valor(es) nulo(s)


In [20]:
analise_pre('heel-height-value',df_teste,0,0)
men_women('heel-height-value',df_teste,'mm')

Valores:
Men: 29mm |  Women: 29mm    108
Men: 28mm |  Women: 28mm    102
Men: 27mm |  Women: 27mm    100
Men: 30mm |  Women: 30mm     97
Men: 25mm |  Women: 25mm     91
Men: 24mm |  Women: 24mm     80
Men: 22mm |  Women: 22mm     77
Name: heel-height-value, dtype: int64
...
 
Existe(m) 781 valor(es) nulo(s)


In [21]:
analise_pre('heel-to-toe-drop-value',df_teste,0,0)
men_women('heel-to-toe-drop-value',df_teste,'mm')

Valores:
Men: 10mm |  Women: 10mm    509
Men: 8mm |  Women: 8mm      430
Men: 4mm |  Women: 4mm      218
Men: 12mm |  Women: 12mm    189
Men: 6mm |  Women: 6mm      188
Men: 0mm |  Women: 0mm      123
Men: 5mm |  Women: 5mm       86
Name: heel-to-toe-drop-value, dtype: int64
...
 
Existe(m) 219 valor(es) nulo(s)


In [22]:
analise_pre('weight-value',df_teste,0,0)
men_women('weight-value',df_teste,'g')

Valores:
Men: 275g |  Women: 275g    13
Men: 255g |  Women: 198g    11
Men: 283g |  Women: 283g    10
Men: 312g                    9
Men: 295g |  Women: 255g     9
Men: 283g |  Women: 227g     9
Men: 290g |  Women: 250g     8
Name: weight-value, dtype: int64
...
 
Existe(m) 150 valor(es) nulo(s)


##     

<a id='t2_4'></a>
## 2.4 - Transformação das variáveis particulares
- [Sumário](#top) 
    - [Anterior](#t2_3)
    - [Próximo](#t2_5)

### Assim como na sessão anterior, aqui todos os nulos foram substituídos por -1.

### Para o preço, foi apenas necessário extrair (com o auxílio da lib **re**) o número que se encontrava após o RS:

In [23]:
analise_pre('price-value',df_teste,0,0)
df_teste['price']=df_teste['price-value'].fillna(-1).apply(lambda row: -1 if row == -1 
                                                           else (int(re.findall('(\d+)',row)[0]) if re.findall('(\d+)',row)!=[] else -1))
df_teste.drop(columns='price-value',inplace=True)

Valores:
R$880     304
R$950     243
R$730     203
R$1100    174
R$810     165
R$1020    132
R$1170    128
Name: price-value, dtype: int64
...
 
Existe(m) 3 valor(es) nulo(s)


### Para tratar o *score* do *review* dos profissionais, buscou-se todos os conjuntos de números presentes no registro. Ao analisar o *output* (em forma de lista) podemos perceber que o primeiro (índice 0) e o ultimo (índice 2) resultado são referentes, respectivamente, ao *score* e a quantas pessoas realizaram aquele *review*:

In [24]:
analise_pre('rr-reviews-score-average',df_teste,0,0)

df_teste['expert_score']=df_teste['rr-reviews-score-average'].fillna(-1).apply(lambda row: -1 if row == -1 
                                                                               else (int(re.findall('(\d+)',row)[0]) if re.findall('(\d+)',row)!=[] else -1))
df_teste['n_expert_reviews']=df_teste['rr-reviews-score-average'].fillna(-1).apply(lambda row: -1 if row == -1 
                                                                                 else (int(re.findall('(\d+)',row)[2]) if re.findall('(\d+)',row)!=[] else -1))
df_teste.drop(columns='rr-reviews-score-average',inplace=True)

Valores:
90 / 100 based on 1 expert reviews    43
80 / 100 based on 1 expert reviews    40
85 / 100 based on 1 expert reviews    22
88 / 100 based on 1 expert reviews    19
84 / 100 based on 2 expert reviews    16
95 / 100 based on 1 expert reviews    15
82 / 100 based on 1 expert reviews    13
Name: rr-reviews-score-average, dtype: int64
...
 
Existe(m) 831 valor(es) nulo(s)


### Os indicadores correspondentes ao número de razões para comprar e não comprar o tênis precisaram ser transformados em inteiros, já que estavam como objetos de texto. Os textos explicativos que estavam nulos foram substituídos por uma *string* vazia:

In [25]:
df_teste['reasons_not_to_buy']=df_teste['reasons_not_to_buy'].fillna(0).apply(lambda x: int(x))
df_teste['reasons_to_buy']=df_teste['reasons_to_buy'].fillna(0).apply(lambda x: int(x))

df_teste['bad_reasons_to_buy']=df_teste['bad_reasons_to_buy'].fillna('')
df_teste['good_reasons_to_buy']=df_teste['good_reasons_to_buy'].fillna('')

### Por fim, foi adicionado o link do site para criar um acesso direto a página do produto:

In [26]:
df_teste['link']='https://runrepeat.com/'+df_teste['link'].copy()

##     

<a id='t2_5'></a>
## 2.5 - Variáveis descartadas devido ao excesso de nulos
- [Sumário](#top) 
    - [Anterior](#t2_4)
    - [Próximo](#t3)

### Durante o processo de exploração, percebemos que existem diversas variáveis com muitos valores nulos. Desse modo, optou-se por utilizar apenas aqueles que tivessem, pelo menos, 40% dos seus dados preenchidos.
### Todas *features* listadas abaixo foram eliminadas, a partir do racional descrito acima.

##     

In [27]:
analise_pre('fit-value',df_teste,1,1)

Valores:
Medium forefoot,  Medium heel,  Medium toe box    262
Medium heel,  Medium toe box                       12
Medium forefoot,  Medium heel,  Wide toe box        7
Narrow forefoot,  Narrow heel,  Narrow toe box      5
Medium forefoot                                     4
Medium forefoot,  Medium heel,  Narrow toe box      4
Wide toe box                                        3
Name: fit-value, dtype: int64
...
 
Existe(m) 1943 valor(es) nulo(s)
 
Temos os seguintes valores nesta variável:
['Medium forefoot', 'Medium heel', 'Medium toe box', 'Wide toe box', 'Narrow forefoot', 'Narrow heel', 'Narrow toe box', 'Wide forefoot', 'Wide heel']
 
Existem 10 possibilidades de features a serem adicionadas


In [28]:
analise_pre('Stability',df_teste,1,1)

Valores:
5/10     127
3/10      32
2/10      20
10/10     19
6/10      13
8/10      11
4/10      10
Name: Stability, dtype: int64
...
 
Existe(m) 2036 valor(es) nulo(s)
 
Temos os seguintes valores nesta variável:
['5/10', '3/10', '2/10', '10/10', '6/10', '8/10', '4/10', '7/10', '9/10']
 
Existem 10 possibilidades de features a serem adicionadas


In [29]:
analise_pre('Comfort',df_teste,1,1)

Valores:
8/10     229
7/10     161
9/10     114
6/10      52
10/10     15
5/10      12
Name: Comfort, dtype: int64
...
 
Existe(m) 1694 valor(es) nulo(s)
 
Temos os seguintes valores nesta variável:
['8/10', '7/10', '9/10', '6/10', '10/10', '5/10']
 
Existem 7 possibilidades de features a serem adicionadas


In [30]:
analise_pre('Durability',df_teste,1,1)

Valores:
8/10     193
9/10     136
7/10     121
10/10     59
6/10      47
5/10      24
4/10       1
Name: Durability, dtype: int64
...
 
Existe(m) 1696 valor(es) nulo(s)
 
Temos os seguintes valores nesta variável:
['8/10', '9/10', '7/10', '10/10', '6/10', '5/10', '4/10']
 
Existem 8 possibilidades de features a serem adicionadas


In [31]:
analise_pre('Traction',df_teste,1,1)

Valores:
7/10     200
8/10     132
9/10     105
10/10     89
6/10      43
5/10      11
Name: Traction, dtype: int64
...
 
Existe(m) 1697 valor(es) nulo(s)
 
Temos os seguintes valores nesta variável:
['7/10', '8/10', '9/10', '10/10', '6/10', '5/10']
 
Existem 7 possibilidades de features a serem adicionadas


In [32]:
analise_pre('material-value',df_teste,1,1)

Valores:
Vegan                                    374
Mesh upper,  Rubber sole                 266
Knit upper,  Rubber sole                  70
Rubber sole                               33
Mesh upper                                29
Mesh upper,  Reflective,  Rubber sole     25
Rubber sole,  Mesh upper                   8
Name: material-value, dtype: int64
...
 
Existe(m) 1381 valor(es) nulo(s)
 
Temos os seguintes valores nesta variável:
['Vegan', 'Mesh upper', 'Rubber sole', 'Knit upper', 'Reflective', 'Vibram sole', 'Leather']
 
Existem 8 possibilidades de features a serem adicionadas


In [33]:
analise_pre('foot-condition-value',df_teste,1,1)

Valores:
Flat feet                                                                                                               22
Plantar fasciitis                                                                                                        7
Foot pain                                                                                                                5
Bunions                                                                                                                  2
Back-pain,  Shin splints                                                                                                 1
Arthritis pain,  Back-pain                                                                                               1
Arthritis pain,  Back-pain,  Bunions,  Hip pain,  Morton's-neuroma,  Plantar fasciitis,  Sesamoiditis,  Shin splints     1
Name: foot-condition-value, dtype: int64
...
 
Existe(m) 2214 valor(es) nulo(s)
 
Temos os seguintes valores nesta variável:
['Fla

In [34]:
analise_pre('technology-value',df_teste,1,1)

Valores:
Gore-Tex                   88
Vibram                     23
BioMoGo                    19
Fresh Foam                 16
AHAR,  Flyte Foam,  Gel    15
Gel                        12
Boost                      10
Name: technology-value, dtype: int64
...
 
Existe(m) 1944 valor(es) nulo(s)
 
Temos os seguintes valores nesta variável:
['Gore-Tex', 'Vibram', 'BioMoGo', 'Fresh Foam', 'AHAR', 'Flyte Foam', 'Gel', 'Boost', 'Nike React', 'HOVR', 'Zoom Air', 'Continental', 'ZoomX', 'Flyknit', 'Flywire', 'AdiWear', 'Fuel Cell', 'SpeedForm', 'Primeknit', '', 'Cloudfoam', 'REVlite', 'carbon fiber plate', 'Ultraboost', 'HYBRID', 'ISOFIT', 'IGNITE', 'NRGY', 'Helion']
 
Existem 30 possibilidades de features a serem adicionadas


In [35]:
analise_pre('release-date-value',df_teste,0,1)

Valores:
Jan 2017    44
Jun 2017    44
Oct 2016    31
Feb 2018    29
Jan 2018    29
Feb 2017    24
Jun 2019    24
Name: release-date-value, dtype: int64
...
 
Existe(m) 1482 valor(es) nulo(s)


In [36]:
analise_pre('features-value',df_teste,0,1)

Valores:
Breathable |  Cushioned |  Comfortable                         286
Slip-on                                                        182
Cushioned |  Comfortable                                        63
Breathable |  Comfortable                                       51
Breathable |  Sockless wear |  Cushioned |  Comfortable         27
Breathable |  Orthotic friendly |  Cushioned |  Comfortable     18
High-top                                                        14
Name: features-value, dtype: int64
...
 
Existe(m) 1553 valor(es) nulo(s)


##     

<a id='t3'></a>
## 3 - Verificação dos resultados
- [Sumário](#top)   
    - [Anterior](#t2_5)

### Após a realização de todos os tratamentos, verificamos atráves da função info( ) do **pandas** se todos os valores são não-nulos e o tipo de cada uma delas.
* Percebe-se que está tudo dentro do esperado ;
* É feita uma verificação visual do **head** e **tail** do *dataset* ;
* Por fim, salvamos o arquivo com os dados tratados, desta vez no formato **csv**.

In [37]:
df_teste.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2277 entries, 0 to 2276
Data columns (total 90 columns):
shoes_name                      2277 non-null object
link                            2277 non-null object
image                           2277 non-null object
reasons_not_to_buy              2277 non-null int64
reasons_to_buy                  2277 non-null int64
bad_reasons_to_buy              2277 non-null object
good_reasons_to_buy             2277 non-null object
terrain_Road                    2277 non-null int64
terrain_Trail                   2277 non-null int64
terrain_Treadmill               2277 non-null int64
terrain_Mud                     2277 non-null int64
terrain_Snow                    2277 non-null int64
arch_Neutral                    2277 non-null int64
arch_Stability                  2277 non-null int64
arch_Motion control             2277 non-null int64
Neutral Pronation               2277 non-null int64
Overpronation                   2277 non-null int64
Seve

In [38]:
df_teste.head(5)

,shoes_name,link,image,reasons_not_to_buy,reasons_to_buy,bad_reasons_to_buy,good_reasons_to_buy,terrain_Road,terrain_Trail,terrain_Treadmill,terrain_Mud,terrain_Snow,arch_Neutral,arch_Stability,arch_Motion control,Neutral Pronation,Overpronation,Severe overpronation,archtype_High arch,archtype_Medium arch,archtype_Low arch,use_Jogging,use_All-day wear,use_Fell running,use_Walking,use_Triathlon,use_Obstacle course racing,brand_Asics,brand_Nike,brand_Adidas,brand_New Balance,brand_Saucony,brand_Reebok,brand_Brooks,brand_Salomon,brand_Under Armour,brand_Mizuno,brand_Puma,brand_Hoka One One,brand_Merrell,brand_Altra,brand_Inov-8,brand_Skechers,brand_Newton,brand_On,brand_The North Face,brand_La Sportiva,brand_Topo Athletic,brand_Zoot,brand_361 Degrees,brand_Vibram FiveFingers,brand_Scott,brand_Dynafit,brand_Vivobarefoot,brand_Jordan,brand_Salming,brand_Arc'teryx,brand_Xero Shoes,brand_Icebug,brand_Columbia,type_Heavy,type_Big guy,type_Low drop,type_Zero drop,type_Maximalist,type_Barefoot,type_Minimalist,width_Normal,width_Wide,width_X-Wide,width_Narrow,strike_Midfoot strike,strike_Heel strike,strike_Forefoot strike,dist_Daily running,dist_Long distance,dist_Marathon,dist_Competition,dist_Ultra running,forefoot-height-value_men,forefoot-height-value_women,heel-height-value_men,heel-height-value_women,heel-to-toe-drop-value_men,heel-to-toe-drop-value_women,weight-value_men,weight-value_women,price,expert_score,n_expert_reviews
0,361 Degrees Chaser 2,https://runrepeat.com//361-degrees-chaser-2?se...,https://cdn.runrepeat.com/i/361-degrees/27865/...,1,6,A few users mentioned that the shoe is too st...,Those who have tried the 361 Degrees Chaser 2...,1,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,15,15,23,23,8,8,232,193,730,92,7
1,361 Degrees Enjector,https://runrepeat.com//361-degrees-enjector?se...,https://cdn.runrepeat.com/i/361-degrees/29213/...,0,0,,,1,0,0,0,0,0,0,1,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,1,1,0,0,-1,-1,-1,-1,8,8,281,230,550,-1,-1
2,361 Degrees Feisu,https://runrepeat.com//361-degrees-feisu?selec...,https://cdn.runrepeat.com/i/361-degrees/28787/...,0,0,,,1,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,17,17,26,26,9,9,201,162,880,84,4
3,361 Degrees Kroozer,https://runrepeat.com//361-degrees-kroozer?sel...,https://cdn.runrepeat.com/i/361-degrees/29214/...,2,8,"One commenter, who prefer flat shoelaces, dis...",A couple of reviewers stated that the 361 Deg...,1,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,1,1,1,0,0,-1,-1,-1,-1,8,8,283,283,300,-1,-1
4,361 Degrees Meraki,https://runrepeat.com//361-degrees-meraki?sele...,https://cdn.runrepeat.com/i/361-degrees/27449/...,2,5,Some testers alleged that the foam midsole wa...,The 361 Degrees Meraki was considered by its ...,1,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,1,0,0,1,1,1,0,0,18,18,27,27,9,9,289,241,950,88,17


In [39]:
df_teste.to_csv('./dados_tratados/dados_final.csv',index=False)

In [40]:
pd.read_csv('./dados_tratados/dados_final.csv').tail()

,shoes_name,link,image,reasons_not_to_buy,reasons_to_buy,bad_reasons_to_buy,good_reasons_to_buy,terrain_Road,terrain_Trail,terrain_Treadmill,terrain_Mud,terrain_Snow,arch_Neutral,arch_Stability,arch_Motion control,Neutral Pronation,Overpronation,Severe overpronation,archtype_High arch,archtype_Medium arch,archtype_Low arch,use_Jogging,use_All-day wear,use_Fell running,use_Walking,use_Triathlon,use_Obstacle course racing,brand_Asics,brand_Nike,brand_Adidas,brand_New Balance,brand_Saucony,brand_Reebok,brand_Brooks,brand_Salomon,brand_Under Armour,brand_Mizuno,brand_Puma,brand_Hoka One One,brand_Merrell,brand_Altra,brand_Inov-8,brand_Skechers,brand_Newton,brand_On,brand_The North Face,brand_La Sportiva,brand_Topo Athletic,brand_Zoot,brand_361 Degrees,brand_Vibram FiveFingers,brand_Scott,brand_Dynafit,brand_Vivobarefoot,brand_Jordan,brand_Salming,brand_Arc'teryx,brand_Xero Shoes,brand_Icebug,brand_Columbia,type_Heavy,type_Big guy,type_Low drop,type_Zero drop,type_Maximalist,type_Barefoot,type_Minimalist,width_Normal,width_Wide,width_X-Wide,width_Narrow,strike_Midfoot strike,strike_Heel strike,strike_Forefoot strike,dist_Daily running,dist_Long distance,dist_Marathon,dist_Competition,dist_Ultra running,forefoot-height-value_men,forefoot-height-value_women,heel-height-value_men,heel-height-value_women,heel-to-toe-drop-value_men,heel-to-toe-drop-value_women,weight-value_men,weight-value_women,price,expert_score,n_expert_reviews
2272,Zoot Ultra Kiawe 2.0,https://runrepeat.com//zoot-ultra-kiawe?select...,https://cdn.runrepeat.com/i/zoot/10224/zoot-ul...,2,5,NaN,This shoe feels comfortable with or without s...,1,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,12,12,18,18,6,6,210,210,810,93,1
2273,Zoot Ultra Race 4.0,https://runrepeat.com//zoot-ultra-race?selecte...,https://cdn.runrepeat.com/i/zoot/10226/zoot-ul...,3,4,NaN,The Ultra Race 4.0 features an UltraFit upper...,1,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,1,1,0,0,13,13,23,23,10,10,281,235,880,82,1
2274,Zoot Ultra TT,https://runrepeat.com//zoot-ultra-tt,https://cdn.runrepeat.com/i/zoot/33731/zoot-ul...,2,7,A couple of users complained because the shoe...,"The Zoot Ultra TT is lightweight, most runner...",1,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,20,20,23,23,3,3,213,184,990,-1,-1
2275,Zoot Ultra TT 7.0,https://runrepeat.com//zoot-ultra-tt-70?select...,https://cdn.runrepeat.com/i/zoot/10230/zoot-ul...,3,6,NaN,Many runners applauded the durable constructi...,1,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,17,17,27,27,10,10,240,226,880,-1,-1
2276,Zoot Ultra Tempo 6.0,https://runrepeat.com//zoot-ultra-tempo,https://cdn.runrepeat.com/i/zoot/10228/zoot-ul...,2,5,NaN,The Ultra Tempo 6.0 provides a sock-like feel...,1,0,0,0,0,0,1,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,1,1,0,0,17,17,27,27,10,10,255,240,880,81,2
